In [1]:
import numpy as np
import geopandas as gpd
import codecs
import requests
import rasterio as rio 

In [2]:
########### USER ###########
#set domain
domain = 'OR'


In [3]:
# path to SnowModel
modpath = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill'+domain+'/'

# path to DEM ascii
DEMpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/DEM_'+domain+'.asc'

#path to SNOTEL gdf
gdfpath = '/nfs/attic/dfh/Aragon2/CSOdata/'+domain+'/CSO_SNOTEL_sites.geojson'

#path to VEG .asc
VEGpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/NLCD2016_'+domain+'.asc'

#path to lat .asc
LATpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/grid_lat.asc'

#path to lon .asc
LONpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/grid_lon.asc'

# VEG outfile path
outVEGpath = modpath+'topo_vege/NLCD2016_line_'+domain+'.asc'

# DEM outfile path
outDEMpath = modpath+'topo_vege/DEM_line_'+domain+'.asc'

# Line outfile path
outFpath = modpath+'extra_met/snowmodel_line_pts.dat'

#lon outfile path
outLONpath = modpath+'extra_met/grid_lon.asc'

#lat outfile path
outLATpath = modpath+'extra_met/grid_lat.asc'

# station data
stn_gdf = gpd.read_file(gdfpath)

#path to CSO domain
domains_resp = requests.get("https://raw.githubusercontent.com/snowmodel-tools/preprocess_python/master/CSO_domains.json")
domains = domains_resp.json()
# CSO projection
mod_proj = domains[domain]['mod_proj']

In [4]:
def getmeta(file_name):
    myvars = {}
    lines = open(file_name, 'r').readlines()
    for i in range(5):
        line=lines[i]
        name, var = line.partition("\t")[::2]
        myvars[name.strip()] = int(var)
    return myvars

In [5]:
myvars = getmeta(DEMpath)
myvars

{'ncols': 821,
 'nrows': 1231,
 'xllcorner': 570400,
 'yllcorner': 4832800,
 'cellsize': 100}

In [6]:
#Build northing easting array 
#Northing
# locate center of lower left cell
st = myvars['yllcorner']+(myvars['cellsize']/2)
#build northing array
north = np.arange(st,st+myvars['nrows']*myvars['cellsize'],myvars['cellsize'])

#Easting
# locate center of lower left cell
st = myvars['xllcorner']+(myvars['cellsize']/2)
#build easting array
east = np.arange(st,st+myvars['ncols']*myvars['cellsize'],myvars['cellsize'])

In [7]:
#fortran indexing starts at 1
#staion 1 to N of N stations
count = np.zeros(stn_gdf.shape[0])
#easting of pixel corresponding to station 
stn_est = np.zeros(stn_gdf.shape[0])
#northing of pixel corresponding to station
stn_nor = np.zeros(stn_gdf.shape[0])
#index of pixel corresponding to station 
est_idx = np.zeros(stn_gdf.shape[0])
#index of pixel corresponding to station 
nor_idx = np.zeros(stn_gdf.shape[0])

for z in range(stn_gdf.shape[0]):
    count[z] = z + 1
    lons = abs(stn_gdf.easting[z]-east)
    loIDX = [i for i, value in enumerate(lons) if value == np.min(abs(stn_gdf.easting[z]-east))] 
    stn_est[z] = east[loIDX[0]]
    est_idx[z] = loIDX[0] + 1
    lats = abs(stn_gdf.northing[z]-north)
    laIDX = [i for i, value in enumerate(lats) if value == np.min(abs(stn_gdf.northing[z]-north))]
    stn_nor[z] = north[laIDX[0]]
    nor_idx[z] = laIDX[0] + 1

In [8]:
print(count)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [9]:
#Print out .dat file 
#https://pyformat.info/
f= open(outFpath,"w+")
for z in range(count.shape[0]):
    f.write('{:08.0f}\t'.format(count[z])+'{:10.0f}\t'.format(est_idx[z])+'{:10.0f}\t'.format(nor_idx[z])+
            '{:10.0f}\t'.format(stn_est[z])+'{:10.0f}\t\n'.format(stn_nor[z]))
f.close() 

## Extract topo, veg, lat, and lon files for snowmodel_line

In [10]:
# function to extract .asc files at SNOTEL locations
def field2line(filepath,outpath):
    with codecs.open(filepath, encoding='utf-8-sig') as f:
        data = np.loadtxt(f,skiprows=6)
    data_line=[]
    for i in range(count.shape[0]):
        info = str(int(data[int(nor_idx[i]-1),int(est_idx[i]-1)]))
        data_line.append(info)
    lines = open(filepath, 'r').readlines()
    head = 'ncols\t1\nnrows\t'+str(count.shape[0])+'\n'+lines[2]+lines[3]+lines[4]+lines[5]
    data = ''
    for s in data_line:
        data += s +'\n'
    f= open(outpath,"w+")
    f.write(head+data)
    f.close()

In [11]:
#extract DEM
field2line(DEMpath,outDEMpath)
#extract VEG
field2line(VEGpath,outVEGpath)
#extract LAT
field2line(LATpath,outLATpath)
#extract LON
field2line(LONpath,outLONpath)

In [ ]:
def build_snowmodel_line(domain):
    # path to SnowModel
    modpath = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill'+domain+'/'

    # path to DEM ascii
    DEMpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/DEM_'+domain+'.asc'

    #path to SNOTEL gdf
    gdfpath = '/nfs/attic/dfh/Aragon2/CSOdata/'+domain+'/CSO_SNOTEL_sites.geojson'

    #path to VEG .asc
    VEGpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/NLCD2016_'+domain+'.asc'

    #path to lat .asc
    LATpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/grid_lat.asc'

    #path to lon .asc
    LONpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/grid_lon.asc'

    # VEG outfile path
    outVEGpath = modpath+'topo_vege/NLCD2016_line_'+domain+'.asc'

    # DEM outfile path
    outDEMpath = modpath+'topo_vege/DEM_line_'+domain+'.asc'

    # Line outfile path
    outFpath = modpath+'extra_met/snowmodel_line_pts.dat'

    #lon outfile path
    outLONpath = modpath+'extra_met/grid_lon.asc'

    #lat outfile path
    outLATpath = modpath+'extra_met/grid_lat.asc'

    # station data
    stn_gdf = gpd.read_file(gdfpath)

    #path to CSO domain
    domains_resp = requests.get("https://raw.githubusercontent.com/snowmodel-tools/preprocess_python/master/CSO_domains.json")
    domains = domains_resp.json()
    # CSO projection
    mod_proj = domains[domain]['mod_proj']

    #get metadata from .asc 
    def getmeta(file_name):
        myvars = {}
        lines = open(file_name, 'r').readlines()
        for i in range(5):
            line=lines[i]
            name, var = line.partition("\t")[::2]
            myvars[name.strip()] = int(var)
        return myvars

    myvars = getmeta(DEMpath)

    #Build northing easting array 
    #Northing
    # locate center of lower left cell
    st = myvars['yllcorner']+(myvars['cellsize']/2)
    #build northing array
    north = np.arange(st,st+myvars['nrows']*myvars['cellsize'],myvars['cellsize'])

    #Easting
    # locate center of lower left cell
    st = myvars['xllcorner']+(myvars['cellsize']/2)
    #build easting array
    east = np.arange(st,st+myvars['ncols']*myvars['cellsize'],myvars['cellsize'])

    #fortran indexing starts at 1
    #staion 1 to N of N stations
    count = np.zeros(stn_gdf.shape[0])
    #easting of pixel corresponding to station 
    stn_est = np.zeros(stn_gdf.shape[0])
    #northing of pixel corresponding to station
    stn_nor = np.zeros(stn_gdf.shape[0])
    #index of pixel corresponding to station 
    est_idx = np.zeros(stn_gdf.shape[0])
    #index of pixel corresponding to station 
    nor_idx = np.zeros(stn_gdf.shape[0])

    for z in range(stn_gdf.shape[0]):
        count[z] = z + 1
        lons = abs(stn_gdf.easting[z]-east)
        loIDX = [i for i, value in enumerate(lons) if value == np.min(abs(stn_gdf.easting[z]-east))] 
        stn_est[z] = east[loIDX[0]]
        est_idx[z] = loIDX[0] + 1
        lats = abs(stn_gdf.northing[z]-north)
        laIDX = [i for i, value in enumerate(lats) if value == np.min(abs(stn_gdf.northing[z]-north))]
        stn_nor[z] = north[laIDX[0]]
        nor_idx[z] = laIDX[0] + 1

    #Print out .dat file 
    f= open(outFpath,"w+")
    for z in range(count.shape[0]):
        f.write('{:08.0f}\t'.format(count[z])+'{:10.0f}\t'.format(est_idx[z])+'{:10.0f}\t'.format(nor_idx[z])+
                '{:10.0f}\t'.format(stn_est[z])+'{:10.0f}\t\n'.format(stn_nor[z]))
    f.close() 

    ## Extract topo, veg, lat, and lon files for snowmodel_line

    # function to extract .asc files at SNOTEL locations
    def field2line(filepath,outpath):
        with codecs.open(filepath, encoding='utf-8-sig') as f:
            data = np.loadtxt(f,skiprows=6)
        data_line=[]
        for i in range(count.shape[0]):
            info = str(int(data[int(nor_idx[i]-1),int(est_idx[i]-1)]))
            data_line.append(info)
        lines = open(filepath, 'r').readlines()
        head = 'ncols\t1\nnrows\t'+str(count.shape[0])+'\n'+lines[2]+lines[3]+lines[4]+lines[5]
        data = ''
        for s in data_line:
            data += s +'\n'
        f= open(outpath,"w+")
        f.write(head+data)
        f.close()

    #extract DEM
    field2line(DEMpath,outDEMpath)
    #extract VEG
    field2line(VEGpath,outVEGpath)
    #extract LAT
    field2line(LATpath,outLATpath)
    #extract LON
    field2line(LONpath,outLONpath)